In [ ]:
!pip install flask Pillow pyngrok

import os, time
from flask import Flask, request, send_file, jsonify
from PIL import Image
from pyngrok import ngrok

# ====== NGROK TOKEN ======
ngrok.set_auth_token("36w8RYbI76MnFBLyFdN52oIjbF2_5rf1c4khfbRRnBnghrKJf")

# ====== CONFIG ======
IMAGE_FOLDER = "images"
WIDTH  = 320
HEIGHT = 240
FRAME_SIZE = WIDTH * HEIGHT * 2  # RGB565

if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

frame_buffer = bytearray()
app = Flask(__name__)

# ====== IMAGE UTILS ======
def rgb565_to_rgb888_one(val):
    r = (val >> 11) & 0x1F
    g = (val >> 5) & 0x3F
    b = val & 0x1F
    return (r * 255 // 31, g * 255 // 63, b * 255 // 31)

def convert_rgb565_to_png(raw_bytes):
    pixels = []
    for i in range(0, len(raw_bytes), 2):
        val = raw_bytes[i] | (raw_bytes[i+1] << 8)
        pixels.append(rgb565_to_rgb888_one(val))

    img = Image.new("RGB", (WIDTH, HEIGHT))
    if len(pixels) == WIDTH * HEIGHT:
        img.putdata(pixels)
    return img

# ====== UPLOAD FROM ESP32 ======
@app.route('/upload_block', methods=['POST'])
def upload_block():
    global frame_buffer
    data = request.data

    if not data:
        return "Empty", 400

    frame_buffer.extend(data)
    print(f"📥 Received {len(data)} bytes | Buffer {len(frame_buffer)}/{FRAME_SIZE}")

    while len(frame_buffer) >= FRAME_SIZE:
        frame = frame_buffer[:FRAME_SIZE]
        frame_buffer = frame_buffer[FRAME_SIZE:]

        img = convert_rgb565_to_png(frame)
        name = f"{IMAGE_FOLDER}/{int(time.time()*1000)}.png"
        img.save(name)
        print(f"✅ Saved {name}")

    return "OK"

# ====== API ======
@app.route('/image/<name>')
def get_image(name):
    return send_file(os.path.join(IMAGE_FOLDER, name), mimetype="image/png")

@app.route('/status')
def status():
    files = sorted(os.listdir(IMAGE_FOLDER))
    return jsonify({
        "count": len(files),
        "latest": files[-1] if files else None
    })

@app.route('/clear', methods=['POST'])
def clear_images():
    print("🔥 CLEAR called from browser")
    for f in os.listdir(IMAGE_FOLDER):
        os.unlink(os.path.join(IMAGE_FOLDER, f))
    return "CLEARED"

# ====== GALLERY ======
@app.route('/')
def gallery():
    files = sorted(os.listdir(IMAGE_FOLDER))
    return f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>FPGA Camera</title>
<style>
body {{ font-family: sans-serif; text-align:center; background:#f0f2f5 }}
.btn {{ padding:10px 20px; border:none; border-radius:5px;
       cursor:pointer; font-weight:bold; margin:5px }}
.clear {{ background:#dc3545; color:white }}
.reload {{ background:#28a745; color:white }}
.card {{ display:inline-block; background:white; padding:10px;
        margin:10px; border-radius:8px }}
#notify {{ display:none; position:fixed; top:20px; right:20px;
          background:#fff3cd; padding:15px; border-radius:5px }}
</style>

<script>
let lastCount = {len(files)};

async function checkStatus() {{
    let r = await fetch('/status');
    let d = await r.json();
    if (d.count > lastCount) {{
        document.getElementById('notify').style.display='block';
        document.getElementById('notify').innerHTML =
            '📸 Có ảnh mới<br>Tải lại sau 2s';
        lastCount = d.count;
        setTimeout(() => location.reload(), 2000);
    }}
}}

async function clearImages() {{
    if (!confirm("Xóa hết ảnh?")) return;
    await fetch('/clear', {{ method: 'POST' }});
    location.reload();
}}

setInterval(checkStatus, 3000);
</script>
</head>

<body>
<div id="notify"></div>
<h1>📸 Ảnh xe vi phạm</h1>

<button class="btn reload" onclick="location.reload()">🔄 Làm mới</button>
<button class="btn clear" onclick="clearImages()">🗑️ Xóa tất cả</button>

<div>
{''.join(f'''
<div class="card">
<div>{f}</div>
<img src="/image/{f}" width="{WIDTH}" height="{HEIGHT}">
</div>
''' for f in reversed(files))}
</div>
</body>
</html>
"""

# ====== RUN ======
ngrok.kill()
url = ngrok.connect(5000)
print("🚀 Server:", url)
app.run(host="0.0.0.0", port=5000)

🚀 Server: NgrokTunnel: "https://polypoid-joette-chromophoric.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 18:51:26] "POST /upload_block HTTP/1.1" 200 -


📥 Received 51200 bytes | Buffer 51200/153600


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 18:51:28] "POST /upload_block HTTP/1.1" 200 -


📥 Received 51200 bytes | Buffer 102400/153600


...

📥 Received 51200 bytes | Buffer 102400/153600


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 22:54:50] "POST /upload_block HTTP/1.1" 200 -


📥 Received 51200 bytes | Buffer 153600/153600
✅ Saved images/1766616890925.png


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 22:55:43] "POST /upload_block HTTP/1.1" 200 -


📥 Received 51200 bytes | Buffer 51200/153600
